<br>
<a href="https://www.nvidia.cn/training/online/">
    <div style="width: 55%; background-color: white; margin-top: 50px;">
    <img src="https://dli-lms.s3.amazonaws.com/assets/general/nvidia-logo.png"
         width="400"
         height="186"
         style="margin: 0px -25px -5px; width: 300px"/>
</a>
<h1 style="line-height: 1.4;"><font color="#76b900"><b>使用 NVIDIA Earth-2 部署 AI 天气模型</h1>
<h2><b>练习 3：</b> 降尺度</h2>
<br>

全球 AI 模型（如 FourCastNet (SFNO)）生成的气象数据通常限制在 0.25° 的网格内，大概对应热带圈的 25 公里分辨率。使用这个网格的主要原因是 ERA5 的庞大数据档案在这个分辨率下很容易获取，并且可以用于大规模的数据驱动模型训练。数值同化和预报系统通常在较低分辨率下运行。例如，全球 ECMWF HRES 预报在 0.1° 网格（大约 10 公里）和立方八面体 (O1280) 网格（大约 9 公里）上可用。区域模型，如天气研究与预报（WRF）模型或 icoasphere 非静水（ICON）模型，通常在 1 公里到 3 公里之间的分辨率上运行。许多应用要求这些公里级甚至亚公里级的分辨率。 

在全球范围内训练和运行高分辨率模型需要大量资源。另一种选择是使用统计降尺度模型。这些模型经过训练，能够将较低分辨率（如 25 公里）转换为更高分辨率（如 2 公里）。统计降尺度是动态降尺度的常见替代方案（即，基于全球输入运行区域数值模型），同样适用于数值天气预报。随着 AI 领域的最新发展，我们现在可以构建更强大的统计降尺度模型。

[CorrDiff](https://arxiv.org/abs/2309.15214) 采用两步法同时将低分辨率数据映射到高分辨率，并合成输入中不存在的新变量。第一步使用 UNet 回归来预测输出场的条件均值。这有助于处理输入和输出之间的显著分布转移，例如风速峰值在网格点之间隐藏。第二步则使用扩散模型恢复物理上合理的表示。扩散模型经过训练，能够迭代去除输入中的噪声，并能揭示回归模型单独无法捕捉到的细节。

![CorrDiff Taiwan](./images/corrdiff.jpg "CorrDiff Taiwan")

课程的第三部分，我们将开发一个工作流，应用在中国台湾省训练的 CorrDiff，针对 FourCastNet (SFNO) 生成的预报进行处理。

In [ ]:
from collections import OrderedDict

import numpy as np
import torch
import xarray as xr
from earth2studio.utils.coords import map_coords, split_coords
from earth2studio.data import fetch_data, GFS
from earth2studio.lexicon import GFSLexicon
from earth2studio.io import KVBackend
from earth2studio.models.dx import CorrDiffTaiwan
from earth2studio.models.px import SFNO
from earth2studio.utils.time import to_time_array
from tqdm import tqdm

from plot import plot_downscaled_forecast, plot_downscaled_samples, plot_downscaling, plot_pop, plot_pop_t2m
from utils import make_quarter_degree

%matplotlib inline

## 推理

覆盖中国台湾省的 CorrDiff 版本提供了用于训练的高分辨率区域气象数据。该模型在 25 公里的网格上处理 12 个变量，并在 2 公里的网格上生成四个变量。它可以通过 Earth2Studio 访问，加载方式和我们之前接触的模型完全相同。

In [ ]:
corrdiff = CorrDiffTaiwan.load_model(CorrDiffTaiwan.load_default_package())

这次，我们从零开始开发一个推理工作流，而不是使用 Earth2Studio 中的预定义工作流。将 FourCastNet (SFNO) 与 CorrDiff-Taiwan 结合起来，FourCastNet (SFNO) 在 25 公里生成预报，而 CorrDiff Taiwan 则将预报降尺度到 2 公里。可以像之前一样加载预报模型。

In [ ]:
fcn = SFNO.load_model(SFNO.load_default_package())

为了加速推理，将模型移至 GPU。

In [ ]:
device = torch.device("cuda")

fcn = fcn.to(device)
corrdiff = corrdiff.to(device)

这次从 GFS 数据初始化预报，而不是 ERA5 数据。这将使得在后期切换到实时预报设置变得容易。预报将覆盖 2024 年 7 月中国台湾省经历热浪的时期。

In [ ]:
gfs = GFS()
start_time = np.datetime64("2024-07-01 12:00:00")

数据检索如之前一样，通过 `fetch_data` 的帮助进行。我们传递数据源、开始时间、输入变量和提前时间（对于输入而言为 0 小时）。再把 GPU 指定为目标设备，以便在开始推理时数据可以直接供模型使用。

In [ ]:
input_coords = fcn.input_coords()

x, coords = fetch_data(
    source=gfs,
    time=to_time_array([start_time]),
    variable=input_coords["variable"],
    lead_time=input_coords["lead_time"],
    device=device
)

接下来，定义要向未来预报的时长。这里将覆盖 12 个 6 小时的时间间隔。请记住，预训练版的 FourCastNet (SFNO) 是以 6 小时为步长生成预报的。

In [ ]:
nsteps = 12
lead_time = np.array([np.timedelta64(6 * i, "h") for i in range(nsteps + 1)])

CorrDiff 是一种扩散模型，因此它可以生成一系列输出场景的分布。这使我们能够从一个低分辨率的输入中创建高分辨率的集成，并获得对小规模天气的概率视角。这里设置每个时间步生成的样本数量为 4。请注意，我们将创建一个确定性预报，仅在降尺度时生成集成。如果想进一步深入，可以在预报和降尺度期间都使用集成。

In [ ]:
corrdiff.number_of_samples = 4

现在，设置数据存储。

In [ ]:
io = KVBackend()

我们提前告诉 IO 后端期望什么类型的数据，这样在推理过程中就可以高效地写入输出文件。对于坐标，提供开始时间（在我们的例子中是一个开始时间）、前置时间（1 个初始条件加上 12 个预报步骤）、样本数量和地理坐标。最后为每个输出变量创建一个数组。

In [ ]:
output_coords = corrdiff.output_coords(corrdiff.input_coords())
io_coords = OrderedDict(
    {
        "time": to_time_array([start_time]),
        "lead_time": lead_time,
        "sample": output_coords["sample"],
        "lat": output_coords["lat"],
        "lon": output_coords["lon"],
    }
)
io.add_array(io_coords, output_coords["variable"])

现在准备好运行推理工作流了。在下面的代码中，用 `map_coords` 来选择 FourCastNet (SFNO) 和 CorrDiff 所需的输入变量。预报由一个迭代器处理，在每个时间步骤立即应用 CorrDiff。

In [ ]:
x, coords = map_coords(x, coords, fcn.input_coords())
fc_iterator = fcn.create_iterator(x, coords)

with tqdm(total=nsteps + 1, desc="Running inference") as pbar:
    for step, (x_i, coords_i) in enumerate(fc_iterator):
        x_i, coords_i = map_coords(x_i, coords_i, corrdiff.input_coords())
        x_i, coords_i = corrdiff(x_i, coords_i)
        io.write(*split_coords(x_i, coords_i))
        pbar.update(1)
        if step == nsteps:
            break

hi_res = io.to_xarray()  # load as xarray Dataset

## 分析

作为对比，我们现在也检索相应的低分辨率 GFS 数据。代码应该和之前的部分很相似。将 `fetch_data`（PyTorch 张量和相应的坐标）的输出转换为 xarray `Dataset` 以方便使用。低分辨率数据仅限于 CorrDiff 中国台湾省覆盖的窗口。

In [ ]:
lo_res = xr.DataArray(
    *fetch_data(
        source=gfs,
        time=to_time_array([start_time]),
        variable=[v for v in output_coords["variable"] if v in GFSLexicon.VOCAB],
        lead_time=hi_res.lead_time.values,
    )
).to_dataset("variable")

# Limit to the window covered by our model
lat_from, lat_to = corrdiff.input_coords()["lat"][[0, -1]]
lon_from, lon_to = corrdiff.input_coords()["lon"][[0, -1]]
lo_res = lo_res.sel(lat=make_quarter_degree(lat_from, lat_to), lon=make_quarter_degree(lon_from, lon_to))

中国台湾省在海峡运营着几个离岸风电场，此外，国内还有陆上风电场。我们可能对这些风电场的预期能源产量感兴趣，而 CorrDiff 可以帮助我们更详细地了解局部风速。下面的图表比较了上排的 GFS 和下排的降尺度 CorrDiff 结果。

In [ ]:
def _get_wind_speed(ds):
    return (ds.u10m ** 2 + ds.v10m ** 2) ** 0.5

plot_downscaling(lo_res.assign(s10m=_get_wind_speed), hi_res.assign(s10m=_get_wind_speed), "s10m", start_time, cb_label="Wind speed (m/s)")

如上所述，CorrDiff 不仅可以为输入变量生成更高分辨率的数据，还可以合成新变量。除了输入数据中的 2 米温度和 10 米风速外，CorrDiff 中国台湾省还计算了 1 小时最大雷达反射率（`mrr`）。雷达反射率是降雨强度的重要代理变量。下面的图表展示了几个时间步的结果。在降雨前的高湿度使 2024 年 7 月经历的热浪对人们造成了特别的不适。

In [ ]:
plot_downscaled_forecast(hi_res, "mrr", start_time, float(lo_res.lon[len(lo_res.lon) // 2]), cb_label="MRR (dBZ)")

我们可以比较不同的集成成员，以区分不确定性较高的区域和不确定性较低的区域。小规模天气的整体模式在样本之间看起来相似，但每个样本都是一个独立的物理现实表现。

In [ ]:
plot_downscaled_samples(hi_res, "mrr", start_time, float(lo_res.lon[len(lo_res.lon) // 2]), cb_label="MRR (dBZ)")

现在让我们更仔细地看看温度。为了让结果更直观，您可以选择使用 °C 还是 °F。

In [ ]:
unit = "°C"

t2m_converters = {
    "°C": lambda ds: ds.t2m - 273.15,
    "°F": lambda ds: (ds.t2m - 273.15) * 9/5 + 32,
    "K": lambda ds: ds
}

lo_res = lo_res.assign(t2m=t2m_converters[unit])
hi_res = hi_res.assign(t2m=t2m_converters[unit])

下面的图表再次比较了 GFS 与 CorrDiff 的结果，这次是 2 米温度的。报告显示，国家低洼地区的温度高达 38°C（100°F）。另一方面，中国台湾省山区的温度在降尺度后明显下降。

In [ ]:
plot_downscaling(lo_res, hi_res, "t2m", start_time, normalize=True, cb_label=f"Temperature [{unit}]")

作为能源消耗的初步指标（比如空调的能耗），现在来看看按地区人口加权的温度。我们根据最近的人口普查结果准备了与 GFS 和 CorrDiff 中国台湾省相应分辨率匹配的权重。可以通过数据平台访问[人口普查数据](https://data.gov.tw/en/datasets/7441)。

In [ ]:
pop_lo = np.load("./data/pop_tw_lo.npy")
msk_lo = (pop_lo > 0).astype(np.float32)
msk_lo /= msk_lo.sum()

pop_hi = np.load("./data/pop_tw_hi.npy")
msk_hi = (pop_hi > 0).astype(np.float32)
msk_hi /= msk_hi.sum()

plot_pop(pop_lo, msk_lo, pop_hi, msk_hi, lo_res.lon, lo_res.lat, hi_res.lon, hi_res.lat, float(lo_res.lon[len(lo_res.lon) // 2]))

现在将权重与温度值相乘，计算出中国台湾省人口所经历的预期温度。

In [ ]:
pop_t2m_gfs = (lo_res.t2m * pop_lo).sum(dim="lat").sum(dim="lon")
msk_t2m_gfs = (lo_res.t2m * msk_lo).sum(dim="lat").sum(dim="lon")

pop_t2m_corrdiff = (hi_res.t2m * pop_hi).sum(dim="ilat").sum(dim="ilon")
msk_t2m_corrdiff = (hi_res.t2m * msk_hi).sum(dim="ilat").sum(dim="ilon")

下面的图表展示了四季天气的结果。昼夜温差显著，白天气温较高。然而，夜晚的温度依然很高，这必然导致了相当大的热压力。没有考虑人口的掩模对应的两条线比基于人口的线要低，这是因为如果不按人口加权，中国台湾省凉爽的山区对结果的影响会更高，而这些地区的居住人口相对较少。CorrDiff 温度生成的结果也稍微低于 GFS 温度。

In [ ]:
plot_pop_t2m(pop_t2m_gfs, msk_t2m_gfs, pop_t2m_corrdiff, msk_t2m_corrdiff, ylabel=f"Temperature [{unit}]")

希望您喜欢我们的 NVIDIA Earth-2 的课程。现在，您可以用 [Earth2Studio](https://github.com/NVIDIA/earth2studio) 构建自己的 AI 气象应用。可以在[用户指南](https://nvidia.github.io/earth2studio/userguide/index.html)中找到更多实践示例。要训练自己的 AI 气象模型，可以查看 [NVIDIA PhysicsNeMo](https://github.com/NVIDIA/physicsnemo/tree/main/examples) 仓库中的示例。